*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Get text and labels for train data
f = open("train-data.tsv", "r")
train_labels = []
train_msgs = []
total_words = ""
for line in f:
  if line[0] == 'h':
    train_labels.append(1)
    train_msgs.append(line[4:].rstrip())
    total_words += line[4:].rstrip() + ' '
  else:
    train_labels.append(0)
    train_msgs.append(line[5:].rstrip())
    total_words += line[5:].rstrip() + ' '
f.close()


In [ ]:
# Get text and labels for test data
f = open("valid-data.tsv", "r")
test_labels = []
test_msgs = []
for line in f:
  if line[0] == 'h':
    test_labels.append(1)
    test_msgs.append(line[4:].rstrip())
    total_words += line[4:].rstrip() + ' '
  else:
    test_labels.append(0)
    test_msgs.append(line[5:].rstrip())
    total_words += line[5:].rstrip() + ' '
f.close()

In [ ]:
total_words

In [ ]:
# Create vocabulary converting words to indexes
unique_words = set(total_words.split(' '))
unique_words.remove('')
vocabulary = {value:key for (key,value) in enumerate(unique_words,1)}
print(vocabulary)

In [ ]:
# Encode train and test messages with vocabulary
MAXLEN = 20
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [vocabulary[word] if word in vocabulary else 0 for word in tokens]
  return keras.preprocessing.sequence.pad_sequences([tokens], MAXLEN)[0]

In [ ]:
# Preprocess data
train_data = []
for msg in train_msgs:
  train_data.append(encode_text(msg))
test_data = []
for msg in test_msgs:
  test_data.append(encode_text(msg))

In [ ]:
print(train_data)
print(test_data)

In [ ]:
# Create model
VOCAB_SIZE = 13484
model = keras.Sequential([
        keras.layers.Embedding(VOCAB_SIZE, 32),
        keras.layers.LSTM(32),
        keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
# Train the model
model.fit(np.array(train_data), np.array(train_labels), epochs = 10, validation_split = 0.2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  encoded_text = encode_text(pred_text)
  pred = np.zeros((1,MAXLEN))
  pred[0] = encoded_text
  prediction = ["ham" if model.predict(pred)>=0.5 else "spam"]
  print(prediction)
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction[0], ans)
    if prediction[0] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
